Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

    Logistic Model with L2 loss
    ----------------------------

In [12]:
batch_size = 128
beta_regularize = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta_regularize*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Now, the graph is constructed with L2 regularization. Now, SGD needs to be run on the graph.

In [13]:
num_steps = 3001

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 49.397758
Minibatch accuracy: 8.6%
Validation accuracy: 11.4%
Minibatch loss at step 500: 0.987028
Minibatch accuracy: 79.7%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 0.743852
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Minibatch loss at step 1500: 0.682755
Minibatch accuracy: 82.8%
Validation accuracy: 80.5%
Minibatch loss at step 2000: 0.636725
Minibatch accuracy: 83.6%
Validation accuracy: 82.3%
Minibatch loss at step 2500: 0.868841
Minibatch accuracy: 78.1%
Validation accuracy: 80.8%
Minibatch loss at step 3000: 0.790609
Minibatch accuracy: 81.2%
Validation accuracy: 81.6%
Test accuracy: 87.5%


    1-Layer Hidden Neural Network
    -----------------------------


In [18]:
batch_size = 128
hidden_nodes = 1024
beta_reg_param = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_a = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  weights_b = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases_a = tf.Variable(tf.zeros([hidden_nodes]))
  biases_b = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, weights_a) + biases_a)
  output_layer = tf.matmul(hidden_layer, weights_b) + biases_b
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(output_layer, tf_train_labels)) + beta_reg_param*(tf.nn.l2_loss(weights_a)+tf.nn.l2_loss(weights_b))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(output_layer)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_a) + biases_a), weights_b)+biases_b)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_a) + biases_a), weights_b)+biases_b)


with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3416.972656
Minibatch accuracy: 11.7%
Validation accuracy: 32.0%
Minibatch loss at step 500: 21.361874
Minibatch accuracy: 81.2%
Validation accuracy: 84.3%
Minibatch loss at step 1000: 0.842125
Minibatch accuracy: 85.9%
Validation accuracy: 84.1%
Minibatch loss at step 1500: 0.671039
Minibatch accuracy: 84.4%
Validation accuracy: 83.6%
Minibatch loss at step 2000: 0.612849
Minibatch accuracy: 89.1%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 0.822830
Minibatch accuracy: 79.7%
Validation accuracy: 84.2%
Minibatch loss at step 3000: 0.721289
Minibatch accuracy: 84.4%
Validation accuracy: 84.4%
Test accuracy: 89.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [16]:
#Constrained batch size
batch_size = 10
beta_regularize = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta_regularize*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 48.065018
Minibatch accuracy: 0.0%
Validation accuracy: 13.9%
Minibatch loss at step 500: 1.724852
Minibatch accuracy: 70.0%
Validation accuracy: 68.8%
Minibatch loss at step 1000: 2.381252
Minibatch accuracy: 70.0%
Validation accuracy: 72.4%
Minibatch loss at step 1500: 1.404009
Minibatch accuracy: 90.0%
Validation accuracy: 68.7%
Minibatch loss at step 2000: 3.308974
Minibatch accuracy: 70.0%
Validation accuracy: 67.3%
Minibatch loss at step 2500: 3.073944
Minibatch accuracy: 60.0%
Validation accuracy: 69.0%
Minibatch loss at step 3000: 2.171704
Minibatch accuracy: 70.0%
Validation accuracy: 69.7%
Test accuracy: 75.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [26]:
batch_size = 128
hidden_nodes = 1024
beta_reg_param = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_a = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  weights_b = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases_a = tf.Variable(tf.zeros([hidden_nodes]))
  biases_b = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_layer = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_a) + biases_a), keep_prob=0.5)
  output_layer = tf.matmul(hidden_layer, weights_b) + biases_b
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(output_layer, tf_train_labels)) + beta_reg_param*(tf.nn.l2_loss(weights_a)+tf.nn.l2_loss(weights_b))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(output_layer)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_a) + biases_a), weights_b)+biases_b)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_a) + biases_a), weights_b)+biases_b)


with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 809.039551
Minibatch accuracy: 10.9%
Validation accuracy: 22.5%
Minibatch loss at step 500: 199.755554
Minibatch accuracy: 74.2%
Validation accuracy: 81.2%
Minibatch loss at step 1000: 118.705269
Minibatch accuracy: 75.8%
Validation accuracy: 78.9%
Minibatch loss at step 1500: 69.937668
Minibatch accuracy: 78.9%
Validation accuracy: 81.6%
Minibatch loss at step 2000: 41.601620
Minibatch accuracy: 80.5%
Validation accuracy: 83.1%
Minibatch loss at step 2500: 25.407660
Minibatch accuracy: 74.2%
Validation accuracy: 84.3%
Minibatch loss at step 3000: 15.602824
Minibatch accuracy: 82.0%
Validation accuracy: 85.8%
Test accuracy: 91.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [50]:
##################################################################################################################
#Parameters which need experimentation
batch_size = 1500
num_nodes = [image_size*image_size, 1024, 1024, 800, num_labels]
dropout_prob = [0.8, 0.8, 0.8]
beta_reg_param = 0.3
start_learning_rate = 0.000005
##################################################################################################################

graph = tf.Graph()

def get_prediction(dataset, weights, biases):
  last_layer = dataset
  for i in range(len(num_nodes)-2):
    last_layer = tf.nn.relu(tf.matmul(last_layer,weights[i])+biases[i])
  output_layer = tf.matmul(last_layer, weights[-1]) + biases[-1]
    
  return tf.nn.softmax(output_layer)
    



In [ ]:
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  with tf.variable_scope("layers", regularizer=tf.contrib.layers.l2_regularizer):
      weights = []
      biases = []
      for i in range(len(num_nodes)-1):
          weights.append(tf.Variable(tf.truncated_normal([num_nodes[i],num_nodes[i+1]])))
          biases.append(tf.Variable(tf.zeros([num_nodes[i+1]])))
            
  global_step = tf.Variable(0, trainable=False)
  
  # Training computation.
  # Have fun creating really deep layers
  last_layer = tf_train_dataset
  for i in range(len(num_nodes)-2):
    last_layer = tf.nn.dropout(tf.nn.relu(tf.matmul(last_layer,weights[i])+biases[i]), keep_prob=dropout_prob[i])
  output_layer = tf.matmul(last_layer, weights[-1]) + biases[-1]
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output_layer, tf_train_labels))
  reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
  loss = loss + beta_reg_param*tf.reduce_sum(reg_losses)
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(
      start_learning_rate, global_step, 60000, 0.96, staircase=True)
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss,global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(output_layer)
  valid_prediction = get_prediction(tf_valid_dataset, weights, biases)
  test_prediction = get_prediction(tf_test_dataset, weights, biases)

num_steps = 100000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 118668.648438
Minibatch accuracy: 9.7%
Validation accuracy: 7.0%
Minibatch loss at step 2000: 144036.921875
Minibatch accuracy: 6.1%
Validation accuracy: 22.4%
Minibatch loss at step 4000: 71521.148438
Minibatch accuracy: 26.3%
Validation accuracy: 43.5%
Minibatch loss at step 6000: 74891.679688
Minibatch accuracy: 25.3%
Validation accuracy: 56.1%
Minibatch loss at step 8000: 68454.125000
Minibatch accuracy: 29.1%
Validation accuracy: 63.8%
Minibatch loss at step 10000: 44450.789062
Minibatch accuracy: 47.2%
Validation accuracy: 68.1%
Minibatch loss at step 12000: 47908.449219
Minibatch accuracy: 43.0%
Validation accuracy: 71.1%
Minibatch loss at step 14000: 36920.214844
Minibatch accuracy: 52.9%
Validation accuracy: 73.0%
Minibatch loss at step 16000: 31958.117188
Minibatch accuracy: 60.9%
Validation accuracy: 74.3%
Minibatch loss at step 18000: 29593.183594
Minibatch accuracy: 64.3%
Validation accuracy: 75.4%
Minibatch loss at step 20000: 39502.5